In [1]:
# Install required packages
!pip install -q datasets huggingface-hub pandas numpy matplotlib seaborn plotly polars

print("Installation complete!")

Installation complete!


In [2]:
import os
import sys
import warnings
import shutil

# Suppress all warnings
warnings.filterwarnings('ignore')

# Disable HuggingFace progress bars
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_VERBOSITY"] = "error"

# Disable datasets progress bars
from datasets.utils import disable_progress_bar, logging as datasets_logging
disable_progress_bar()
datasets_logging.set_verbosity_error()

# Disable transformers logging
import transformers
transformers.logging.set_verbosity_error()

# Disable tqdm globally
from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

try:
    from rich import progress
    progress.Progress = lambda *args, **kwargs: None
except:
    pass

# Disable huggingface_hub progress
try:
    from huggingface_hub import logging as hf_logging
    hf_logging.set_verbosity_error()
except:
    pass

print(" All progress bars disabled")


 All progress bars disabled


In [3]:
# Imports
import pandas as pd
import numpy as np
import polars as pl
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("All imports successful!")

All imports successful!


In [4]:
from datasets import load_dataset
dataset = load_dataset("calmgoose/amazon-product-data-2020", split="train")

print(f"Dataset loaded!")

Dataset loaded!


In [5]:
# Convert to pandas DataFrame
df_raw = dataset.to_pandas()

print(f"Converted to pandas DataFrame")
print(f"Shape: {df_raw.shape}")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Converted to pandas DataFrame
Shape: (10002, 15)
Memory usage: 27.69 MB


In [6]:
# Define columns to keep
columns_to_keep = [
    'Uniq Id',
    'Product Name',
    'Category',
    'Selling Price',
    'About Product',
    'Product Specification',
    'Technical Details',
    'Image',
    'Product Url',
    'Is Amazon Seller'
]

# Create cleaned dataframe
df = df_raw[columns_to_keep].copy()

print("Columns dropped successfully")
print(f"Original shape: {df_raw.shape}")
print(f"New shape: {df.shape}")
print(f"Columns removed: {df_raw.shape[1] - df.shape[1]}")
print(f"\nRemaining columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

Columns dropped successfully
Original shape: (10002, 15)
New shape: (10002, 10)
Columns removed: 5

Remaining columns:
  1. Uniq Id
  2. Product Name
  3. Category
  4. Selling Price
  5. About Product
  6. Product Specification
  7. Technical Details
  8. Image
  9. Product Url
  10. Is Amazon Seller


In [7]:
# Check missing values in our kept columns
print("Missing values analysis after column drop:")
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Pct': (df.isnull().sum().values / len(df) * 100).round(2)
})
print(missing_analysis.to_string(index=False))

Missing values analysis after column drop:
               Column  Missing_Count  Missing_Pct
              Uniq Id              0         0.00
         Product Name              0         0.00
             Category            830         8.30
        Selling Price            107         1.07
        About Product            273         2.73
Product Specification           1632        16.32
    Technical Details            790         7.90
                Image              0         0.00
          Product Url              0         0.00
     Is Amazon Seller              0         0.00


In [8]:
# Check rows with critical missing data
print("\nCritical missing data analysis:")
print(f"Missing price: {df['Selling Price'].isna().sum()}")
print(f"Missing category: {df['Category'].isna().sum()}")
print(f"Missing title: {df['Product Name'].isna().sum()}")
print(f"Missing ALL text fields (description + specifications + details): {((df['About Product'].isna()) & (df['Product Specification'].isna()) & (df['Technical Details'].isna())).sum()}")


Critical missing data analysis:
Missing price: 107
Missing category: 830
Missing title: 0
Missing ALL text fields (description + specifications + details): 57


In [9]:
# Store original size
original_size = len(df)
print(f"Starting with {original_size} products")

# Drop rows missing price
df = df[df['Selling Price'].notna()].copy()
print(f"After dropping missing price: {len(df)} products (dropped {original_size - len(df)})")

# Drop rows missing category
before_cat = len(df)
df = df[df['Category'].notna()].copy()
print(f"After dropping missing category: {len(df)} products (dropped {before_cat - len(df)})")

# Drop rows missing ALL text fields
before_text = len(df)
df = df[~((df['About Product'].isna()) & (df['Product Specification'].isna()) & (df['Technical Details'].isna()))].copy()
print(f"After dropping products with no text: {len(df)} products (dropped {before_text - len(df)})")

# Summary
total_dropped = original_size - len(df)
print(f"\nTotal products dropped: {total_dropped} ({total_dropped/original_size*100:.2f}%)")
print(f"Final dataset size: {len(df)} products ({len(df)/original_size*100:.2f}% retained)")

Starting with 10002 products
After dropping missing price: 9895 products (dropped 107)
After dropping missing category: 9066 products (dropped 829)
After dropping products with no text: 9010 products (dropped 56)

Total products dropped: 992 (9.92%)
Final dataset size: 9010 products (90.08% retained)


In [10]:
# Reset index after dropping rows
df = df.reset_index(drop=True)
print(f"Index reset. Final shape: {df.shape}")

Index reset. Final shape: (9010, 10)


In [11]:
df = df.rename(columns={
    'Uniq Id': 'product_id',
    'Product Name': 'title',
    'Category': 'category',
    'Selling Price': 'price',
    'About Product': 'description',
    'Product Specification': 'specifications',
    'Technical Details': 'details',
    'Image': 'image_url',
    'Product Url': 'product_url',
    'Is Amazon Seller': 'is_amazon_seller'
})

print("Columns renamed")
print(df.columns.tolist())

Columns renamed
['product_id', 'title', 'category', 'price', 'description', 'specifications', 'details', 'image_url', 'product_url', 'is_amazon_seller']


In [12]:
df['price'] = df['price'].str.replace('$', '').str.replace(',', '')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

print("Price cleaned")
print(f"Valid prices: {df['price'].notna().sum()}")
print(f"Range: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
print(f"Mean: ${df['price'].mean():.2f}")
print(f"Median: ${df['price'].median():.2f}")

Price cleaned
Valid prices: 8663
Range: $0.01 - $5332.00
Mean: $40.31
Median: $16.99


In [13]:
df['category_split'] = df['category'].str.split(' | ')

df['main_category'] = df['category_split'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df['sub_category'] = df['category_split'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
df['leaf_category'] = df['category_split'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else None)

# Fix trailing commas
df['main_category'] = df['main_category'].str.rstrip(',').str.strip()

df['main_category_norm'] = df['main_category'].str.lower().str.replace(' ', '_').str.replace('&', 'and')
df['sub_category_norm'] = df['sub_category'].str.lower().str.replace(' ', '_').str.replace('&', 'and')

df = df.drop(columns=['category_split'])

print("Categories parsed")
print(f"Unique main: {df['main_category'].nunique()}")
print(f"\nTop 10:")
print(df['main_category'].value_counts())

Categories parsed
Unique main: 22

Top 10:
main_category
Toys           6560
Home            702
Clothing        629
Sports          499
Baby            214
Arts            120
Office           70
Remote           39
Hobbies          34
Industrial       29
Health           22
Tools            17
Pet              16
Patio            12
Grocery          11
Beauty           11
Automotive        9
Electronics       7
Video             5
Musical           2
Movies            1
Cell              1
Name: count, dtype: int64


In [14]:
import re

def deep_clean_text(text):
    if pd.isna(text):
        return None

    text = str(text)

    # Remove boilerplate first
    boilerplate_phrases = [
        'Make sure this fits by entering your model number',
        'Make sure this fits by entering your model number.',
    ]
    for phrase in boilerplate_phrases:
        text = text.replace(phrase, '')

    # Remove emojis and special unicode
    text = text.encode('ascii', 'ignore').decode('ascii')

    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)

    # Remove ". |" artifacts
    text = re.sub(r'\.\s*\|', '|', text)
    text = re.sub(r'\|\s*\.', '|', text)

    # Remove standalone periods and dots
    text = re.sub(r'\s+\.\s+', ' ', text)
    text = re.sub(r'^\.\s*', '', text)

    # Remove repeated punctuation
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\|{2,}', '|', text)

    # Strip
    text = text.strip()
    text = text.strip('|').strip()

    # Return None if empty or just punctuation
    if text in ['', '.', '..', '...']:
        return None

    return text

# Apply to all text columns
print("Deep cleaning text columns...")
df['description'] = df['description'].apply(deep_clean_text)
df['specifications'] = df['specifications'].apply(deep_clean_text)
df['details'] = df['details'].apply(deep_clean_text)

print("Text cleaning complete")
print(f"Description: {df['description'].notna().sum()}")
print(f"Specifications: {df['specifications'].notna().sum()}")
print(f"Details: {df['details'].notna().sum()}")

Deep cleaning text columns...
Text cleaning complete
Description: 8813
Specifications: 7497
Details: 8314


In [15]:
# Now recreate embedding_text with cleaned source columns
def create_clean_embedding_text(row):
    parts = []

    if pd.notna(row['title']):
        parts.append(str(row['title']))

    if pd.notna(row['description']):
        parts.append(str(row['description']))

    if pd.notna(row['specifications']):
        parts.append(str(row['specifications']))

    if pd.notna(row['details']):
        parts.append(str(row['details']))

    return ' | '.join(parts) if parts else None

df['embedding_text'] = df.apply(create_clean_embedding_text, axis=1)
df['text_length'] = df['embedding_text'].str.len()

print("\nEmbedding text recreated")
print(f"Avg length: {df['text_length'].mean():.0f} chars")
print(f"Median: {df['text_length'].median():.0f} chars")


Embedding text recreated
Avg length: 1282 chars
Median: 1140 chars


In [16]:
# Show samples of cleaned text
print("Cleaned samples (first 200 chars):")
for i in range(5):
    print(f"\n{i+1}. {df['embedding_text'].iloc[i][:200]}...")

Cleaned samples (first 200 chars):

1. DB Longboards CoreFlex Crossbow 41" Bamboo Fiberglass Longboard Complete | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex patt...

2. Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5) | Snap circuits mini kits classpack provides basic electronic circuitry activities for students in grades ...

3. 3Doodler Create Flexy 3D Printing Filament Refill Bundle (X5 Pack, Over 1000'. of Extruded Plastics! - Innovate | Smooth 3D drawing experienced the best 3D drawing experience by only using 3Doodler Cr...

4. Guillow Airplane Design Studio with Travel Case Building Kit | Make 8 different Planes at one time| Experiment with different designs and learn about flight| All contained in Part trays inside an attr...

5. Woodstock- Collage 500 pc Puzzle | Puzzle has 500 pieces | Completed puzzle measure 14 x 19 | 100% officially licensed merchandis

In [17]:
def extract_brand(title):
    if pd.isna(title):
        return None
    words = str(title).split()
    if len(words) > 0:
        return words[0]
    return None

df['brand'] = df['title'].apply(extract_brand)

print(f"Brands: {df['brand'].nunique()}")
print(f"\nTop 10:")
print(df['brand'].value_counts().head(10))

Brands: 2406

Top 10:
brand
Rubie's        222
MightySkins    183
Melissa        133
Disney         120
The            119
Forum          107
Amscan         107
Funko          103
amscan         100
Redcat          78
Name: count, dtype: int64


In [18]:
materials = ['steel', 'stainless', 'plastic', 'wood', 'wooden', 'metal', 'aluminum',
             'glass', 'ceramic', 'bamboo', 'cotton', 'polyester', 'leather', 'rubber', 'silicone']

def extract_materials(text):
    if pd.isna(text):
        return []
    text_lower = str(text).lower()
    found = [mat for mat in materials if mat in text_lower]
    return found if found else None

df['materials'] = df['embedding_text'].apply(extract_materials)
df['has_material'] = df['materials'].notna()

print(f"With materials: {df['has_material'].sum()}")

With materials: 3370


In [19]:
import re

eco_keywords = ['eco-friendly', 'organic', 'sustainable', 'biodegradable',
                'recyclable', 'natural', 'green', 'environmental', 'plant-based',
                'eco friendly', 'earth friendly']

def is_eco_friendly(text):
    if pd.isna(text):
        return False
    text_lower = str(text).lower()
    for keyword in eco_keywords:
        if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
            return True
    return False

df['eco_friendly'] = df['embedding_text'].apply(is_eco_friendly)

print(f"Eco-friendly: {df['eco_friendly'].sum()} ({df['eco_friendly'].sum()/len(df)*100:.2f}%)")

Eco-friendly: 908 (10.08%)


In [20]:
def price_bucket(price):
    if pd.isna(price):
        return None
    if price < 15:
        return 'budget'
    elif price < 50:
        return 'mid'
    else:
        return 'premium'

df['price_bucket'] = df['price'].apply(price_bucket)
df['has_description'] = df['description'].notna()
df['has_specifications'] = df['specifications'].notna()
df['has_details'] = df['details'].notna()
df['has_image'] = df['image_url'].notna()
df['doc_id'] = 'doc_' + df.index.astype(str).str.zfill(5)

print("Metadata created")
print(f"\nPrice buckets:")
print(df['price_bucket'].value_counts())

Metadata created

Price buckets:
price_bucket
budget     3892
mid        3580
premium    1191
Name: count, dtype: int64


In [21]:
before = len(df)

df = df[df['price'] >= 0.5].copy()
print(f"Removed {before - len(df)} products with price < $0.50")

df = df.reset_index(drop=True)
df['doc_id'] = 'doc_' + df.index.astype(str).str.zfill(5)

print(f"Final size: {len(df)} products")

Removed 349 products with price < $0.50
Final size: 8661 products


In [22]:
print("Final Dataset")
print(f"Products: {len(df)}")
print(f"Columns: {len(df.columns)}")
print(f"\nKey stats:")
print(f"Price: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
print(f"Categories: {df['main_category'].nunique()}")
print(f"Avg text: {df['text_length'].mean():.0f} chars")
print(f"Eco: {df['eco_friendly'].sum()}")
print(f"\nColumns:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

Final Dataset
Products: 8661
Columns: 27

Key stats:
Price: $0.51 - $5332.00
Categories: 22
Avg text: 1295 chars
Eco: 868

Columns:
 1. product_id
 2. title
 3. category
 4. price
 5. description
 6. specifications
 7. details
 8. image_url
 9. product_url
10. is_amazon_seller
11. main_category
12. sub_category
13. leaf_category
14. main_category_norm
15. sub_category_norm
16. embedding_text
17. text_length
18. brand
19. materials
20. has_material
21. eco_friendly
22. price_bucket
23. has_description
24. has_specifications
25. has_details
26. has_image
27. doc_id


In [23]:
print("Sample of cleaned products:")
print(df.sample(5).to_string(index=False))

Sample of cleaned products:
                      product_id                                                                                                                                                         title                                                                     category  price                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [37]:
print("Embedding text samples (first 300 chars):")
for i in range(5):
    sample = df.iloc[i]
    print(f"\nProduct {i+1}: {sample['title'][:50]}...")
    print(f"Text length: {sample['text_length']} chars")
    print(f"Embedding preview: {sample['embedding_text'][:300]}...")

Embedding text samples (first 300 chars):

Product 1: DB Longboards CoreFlex Crossbow 41" Bamboo Fibergl...
Text length: 1288 chars
Embedding preview: DB Longboards CoreFlex Crossbow 41" Bamboo Fiberglass Longboard Complete | RESPONSIVE FLEX: The Crossbow features a bamboo core encased in triaxial fiberglass and HD plastic for a responsive flex pattern thats second to none. Pumping & carving have never been so satisfying! Flex 2 is recommended for...

Product 2: Electronic Snap Circuits Mini Kits Classpack, FM R...
Text length: 2293 chars
Embedding preview: Electronic Snap Circuits Mini Kits Classpack, FM Radio, Motion Detector, Music Box (Set of 5) | Snap circuits mini kits classpack provides basic electronic circuitry activities for students in grades 2-6 | Includes 5 separate mini building kits- an FM radio, a motion detector, music box, space battl...

Product 3: 3Doodler Create Flexy 3D Printing Filament Refill ...
Text length: 2032 chars
Embedding preview: 3Doodler Create Flexy 3

In [36]:
print("Missing values in final dataset:")
missing = df.isnull().sum()
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': (missing.values / len(df) * 100).round(2)
})
print(missing_df[missing_df['Missing'] > 0].to_string(index=False))

Missing values in final dataset:
        Column  Missing  Percent
   description      175     2.02
specifications     1358    15.68
       details      634     7.32
     materials     5445    62.87


In [35]:
print("Price distribution check:")
print(f"Min: ${df['price'].min():.2f}")
print(f"25th percentile: ${df['price'].quantile(0.25):.2f}")
print(f"Median: ${df['price'].median():.2f}")
print(f"75th percentile: ${df['price'].quantile(0.75):.2f}")
print(f"95th percentile: ${df['price'].quantile(0.95):.2f}")
print(f"Max: ${df['price'].max():.2f}")

Price distribution check:
Min: $0.51
25th percentile: $9.99
Median: $16.99
75th percentile: $30.00
95th percentile: $125.50
Max: $5332.00


In [34]:
print("Category distribution check:")
category_pct = (df['main_category'].value_counts() / len(df) * 100).round(2)
print(category_pct.head(10))
print(f"\nToys dominate: {category_pct.iloc[0]:.1f}% of dataset")
print("This is expected from Amazon 2020 data")

Category distribution check:
main_category
Toys          74.44
Home           7.83
Clothing       5.90
Sports         4.84
Baby           2.44
Arts           1.35
Office         0.79
Remote         0.45
Hobbies        0.39
Industrial     0.32
Name: count, dtype: float64

Toys dominate: 74.4% of dataset
This is expected from Amazon 2020 data


In [31]:
print("Text length distribution:")
print(f"Min: {df['text_length'].min()} chars")
print(f"10th percentile: {df['text_length'].quantile(0.10):.0f} chars")
print(f"25th percentile: {df['text_length'].quantile(0.25):.0f} chars")
print(f"Median: {df['text_length'].median():.0f} chars")
print(f"75th percentile: {df['text_length'].quantile(0.75):.0f} chars")
print(f"90th percentile: {df['text_length'].quantile(0.90):.0f} chars")
print(f"Max: {df['text_length'].max()} chars")
print(f"\nProducts with very long text (>5000 chars): {(df['text_length'] > 5000).sum()}")

Text length distribution:
Min: 35 chars
10th percentile: 508 chars
25th percentile: 787 chars
Median: 1152 chars
75th percentile: 1649 chars
90th percentile: 2272 chars
Max: 15036 chars

Products with very long text (>5000 chars): 9


In [32]:
print("Feature flags summary:")
print(f"Has description: {df['has_description'].sum()} ({df['has_description'].sum()/len(df)*100:.1f}%)")
print(f"Has specifications: {df['has_specifications'].sum()} ({df['has_specifications'].sum()/len(df)*100:.1f}%)")
print(f"Has details: {df['has_details'].sum()} ({df['has_details'].sum()/len(df)*100:.1f}%)")
print(f"Has material info: {df['has_material'].sum()} ({df['has_material'].sum()/len(df)*100:.1f}%)")
print(f"Eco-friendly: {df['eco_friendly'].sum()} ({df['eco_friendly'].sum()/len(df)*100:.1f}%)")

Feature flags summary:
Has description: 8486 (98.0%)
Has specifications: 7303 (84.3%)
Has details: 8027 (92.7%)
Has material info: 3216 (37.1%)
Eco-friendly: 868 (10.0%)


In [33]:
print("Doc ID validation:")
print(f"Unique doc_ids: {df['doc_id'].nunique()}")
print(f"Total products: {len(df)}")
print(f"All unique: {df['doc_id'].nunique() == len(df)}")
print(f"\nSample doc_ids:")
print(df['doc_id'].head(10).tolist())

Doc ID validation:
Unique doc_ids: 8661
Total products: 8661
All unique: True

Sample doc_ids:
['doc_00000', 'doc_00001', 'doc_00002', 'doc_00003', 'doc_00004', 'doc_00005', 'doc_00006', 'doc_00007', 'doc_00008', 'doc_00009']


In [38]:
from pathlib import Path
Path('data/processed').mkdir(parents=True, exist_ok=True)

df.to_parquet('data/processed/products_cleaned.parquet', index=False)
df.to_csv('data/processed/products_cleaned.csv', index=False)

print("Dataset saved")
print(f"Products: {len(df):,}")
print(f"Columns: {len(df.columns)}")
print("Ready for embedding generation!")

Dataset saved
Products: 8,661
Columns: 27
Ready for embedding generation!
